In [4]:
!unzip /content/drive/MyDrive/vehicle.zip -d /content/drive/MyDrive

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/data/vehicles/4414.png  
  inflating: /content/drive/MyDrive/data/vehicles/4415.png  
  inflating: /content/drive/MyDrive/data/vehicles/4416.png  
  inflating: /content/drive/MyDrive/data/vehicles/4417.png  
  inflating: /content/drive/MyDrive/data/vehicles/4418.png  
  inflating: /content/drive/MyDrive/data/vehicles/4419.png  
  inflating: /content/drive/MyDrive/data/vehicles/442.png  
  inflating: /content/drive/MyDrive/data/vehicles/4420.png  
  inflating: /content/drive/MyDrive/data/vehicles/4421.png  
  inflating: /content/drive/MyDrive/data/vehicles/4422.png  
  inflating: /content/drive/MyDrive/data/vehicles/4423.png  
  inflating: /content/drive/MyDrive/data/vehicles/4424.png  
  inflating: /content/drive/MyDrive/data/vehicles/4425.png  
  inflating: /content/drive/MyDrive/data/vehicles/4426.png  
  inflating: /content/drive/MyDrive/data/vehicles/4427.png  
  inflating: /content/drive/MyDrive

In [5]:
import os

import numpy as np

import shutil

rootdir= '/content/drive/MyDrive/data' #path of the original folder

classes = ['non-vehicles', 'vehicles']

for i in classes:
  os.makedirs(rootdir +'/train/' + i)

  os.makedirs(rootdir +'/test/' + i)

  source = rootdir + '/' + i

  allFileNames = os.listdir(source)

  np.random.shuffle(allFileNames)

  test_ratio = 0.20

  train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                        [int(len(allFileNames)* (1 - test_ratio))])

  train_FileNames = [source+'/'+ name for name in train_FileNames.tolist()]
  test_FileNames = [source+'/' + name for name in test_FileNames.tolist()]

  for name in train_FileNames:
    
    shutil.copy(name, rootdir +'/train/' + i)

  for name in test_FileNames:
    shutil.copy(name, rootdir +'/test/' + i)

In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications import InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [224, 224] 

train_path = '/content/drive/MyDrive/data/train'
valid_path = '/content/drive/MyDrive/data/test'
folders=glob( '/content/drive/MyDrive/data/train/*')
print(folders)

['/content/drive/MyDrive/data/train/non-vehicles', '/content/drive/MyDrive/data/train/vehicles']


In [3]:


resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)



94773248/94765736 [==============================] - 1s 0us/step


In [4]:
for layer in resnet.layers:
  layer.trainable=False

In [5]:
x=Flatten()(resnet.output)

prediction=Dense(len(folders),activation='softmax')(x)


model = Model(inputs=resnet.input, outputs=prediction)

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [7]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [8]:


from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)



In [10]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/data/test',
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            class_mode = 'categorical')



Found 14207 images belonging to 2 classes.
Found 3553 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
202/222 [==========================>...] - ETA: 6:06 - loss: 1.9199 - accuracy: 0.6556

In [ ]:
from tensorflow.keras.models import load_model

model.save('vehicle_res.h5')

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')